In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [1]:
import os
import pandas as pd
import numpy as np

# 설정값
folder = r"C:\Users\GCU\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\5. Final stage\250714_CR 데이터 b-d\csv파일\두께별"
file_name_str = "Cut_7x7_12 Cut_7x7_25 Cut_7x7_50"
gap_value = 0.2  # 20% 빈 공간
output_path = os.path.join(folder, "00Merge.csv")

def merge_data(file_name_str, folder, output_path):
    file_names = file_name_str.split()
    file_paths = [os.path.join(folder, f"{name}.csv") for name in file_names]
    
    dataframes = [pd.read_csv(fp, header=None) for fp in file_paths]
    n_rows = len(dataframes[0])
    time_step = dataframes[0].iloc[1, 0] - dataframes[0].iloc[0, 0]  # 첫 번째 파일 기준 시간 간격
    gap = int(gap_value * n_rows)
    
    total_rows = n_rows + (len(dataframes) - 1) * (n_rows + gap)
    n_cols = len(dataframes) + 1
    result_array = np.full((total_rows, n_cols), np.nan)
    
    # 새 시간 축 생성 (0초부터 시작)
    result_array[:, 0] = np.arange(0, total_rows) * time_step
    
    # 데이터 병합
    row_pos = 0
    for i, df in enumerate(dataframes):
        result_array[row_pos:row_pos + n_rows, i + 1] = df.iloc[:, 1].values  # 전압 값만 사용
        row_pos += n_rows + gap

    # 저장
    pd.DataFrame(result_array).to_csv(output_path, index=False, header=False, float_format="%.6f")
    print(f"✅ 병합된 파일이 저장되었습니다: {output_path}")

# 실행
merge_data(file_name_str, folder, output_path)


✅ 병합된 파일이 저장되었습니다: C:\Users\GCU\OneDrive - gachon.ac.kr\HI Lab\0. Projects\0. On going\1. ionic DC-TENG\0. 실험자료\5. Final stage\250714_CR 데이터 b-d\csv파일\두께별\00Merge.csv
